In [161]:
#Imports
from sgp4.api import jday
import datetime
import pandas as pd
from sgp4.api import Satrec
from sgp4 import omm
import numpy as np
%matplotlib notebook
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
from sklearn.neighbors import KernelDensity
import sklearn.mixture as mix


In [162]:
#Get date
current_time = datetime.datetime.now()
xd, xr = jday(current_time.year , current_time.month , current_time.day, current_time.hour, current_time.minute , current_time.second)
print(xd)
print(xr)

2460386.5
0.5196527777777777


In [163]:
with open('../Data/satellitedata.csv') as f:
    lines = omm.parse_csv(f)
    fields = [field for field in lines]
 


Satellite = []
result_satellite = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    if any([abs(coord) > 10_000 for coord in r]):
     continue
    Satellite.append(r)
    result_satellite.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    
  

data_satellite = np.array(Satellite)
data_satellite = data_satellite[~np.isnan(data_satellite).any(axis=1)]

result_satellite = pd.DataFrame(result_satellite,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2],
                                   mode='markers',name="Satellite", marker={"size":12, "color":"white"})])



fig.show()

# Відстань між всіма супутниками і сміттям +
# Скільки до кожного супутника потрапило спостережень 

# 2-3 точки спільні ми їх заморожуємо, перестаємо обчислювати
# Попробувати зобразити формула норми вектора
# Кожну відстань ділити на норму
#



#Пілся заморозки знов прорахувати відстань 

# 1 варіант знов поставити перевірку 

# 2 варіант 


# k-means method+

# нормування на одиничну гіперкулю+

In [164]:
normalization = np.linalg.norm(data_satellite, axis=1)[:, np.newaxis]
data_satellite_norm = data_satellite / normalization
# data_satellite_norm = data_satellite_norm * normalization
# Створити csv з x_normalization і так далі
# На фронт конвертувати назад нормальні дані не нормалізовані
fig = go.Figure(data=[go.Scatter3d(x=data_satellite_norm[:,0], y=data_satellite_norm[:,1], z=data_satellite_norm[:,2],
                                   mode='markers',name="Satellite", marker={"size":10, "color":"white"})])

fig.show()

In [165]:
# DBSCAN for cosmos garbage
with open('../Data/cosmos.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Cosmos_G = []
result_cosmos = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Cosmos_G.append(r)
    result_cosmos.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])

data_garbage_cosmos = np.array(Cosmos_G)
data_garbage_cosmos = data_garbage_cosmos[~np.isnan(data_garbage_cosmos).any(axis=1)]

result_cosmos = pd.DataFrame(result_cosmos,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers')])
fig.show()


model = DBSCAN(eps=800, min_samples=4)
model.fit_predict(data_garbage_cosmos)
predd = model.fit_predict(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":predd})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)


number of cluster found: 45
cluster for each point:  [-1  0 -1  1 -1 -1  2 -1 35  8 -1 -1 32  3  4 -1 36 -1  5 -1 -1 25 -1 -1
  3 -1 -1 -1 -1  2 -1  3  6  7 -1 -1  5 -1  2  3  3 -1 -1  9  2 -1  8  2
 -1  8 -1  5 -1  2 -1 -1  9 10  5 -1  8 30 -1 42 -1 -1 -1 -1  2 -1  8  8
  2  9 -1 11 12  2  0  8 13  3  2 40 -1  6  2  8  8 14 -1  9 19  8 14 15
 -1  8  8 -1  9 18  8 43 -1 -1  2 16 10 -1  2  9 -1 -1 -1 -1  2  2 16 16
  8  8  9  2 29 -1 -1 -1  9 -1  8 -1  9 -1  8 -1 42 -1 14 -1  8  2 16 -1
  8  9 17 22 16 -1 11  9 -1 -1 -1 18  5 -1 21 19 -1 -1  9  2  8 20  2  5
  8  2  0 -1  5 18 21  2 18 -1 22 15 -1 16 -1  6 14 -1 40 15 20 31 -1  5
 10  5 -1  9  9 -1 -1 23  6  8 -1 24 13  8 32  8 25 -1  3 26 -1 -1  8  8
  9 -1  8 -1 27 -1 -1 -1 20 14  4 28  9 29 -1 -1 -1 -1  8 12 30 18  2 -1
  9 -1  1  5  2 14  9  2 -1 26 27 20  8 -1 -1 16  8  8 -1 42  2 -1 16 -1
 -1  9  5 -1 -1 43 -1  8 12 -1 10 -1  8 31  3  2  3 -1 -1 14 17 -1 26 -1
  8 -1 41 -1 32 -1  8 -1 -1  2 22 24 -1 14  2 -1 -1 -1 30 21  8 19  9  

In [166]:
# DBSCAN for iridium garbage
with open('../Data/iridium.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Iridium_G = []
result_iridium = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Iridium_G.append(r)
    result_iridium.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    

data_garbage_iridium = np.array(Iridium_G)
data_garbage_iridium = data_garbage_iridium[~np.isnan(data_garbage_iridium).any(axis=1)]


result_iridium = pd.DataFrame(result_iridium,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()


fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers')])
fig.show()

model = DBSCAN(eps=1570, min_samples=4)
model.fit_predict(data_garbage_iridium)
pred = model.fit_predict(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":pred})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

number of cluster found: 8
cluster for each point:  [ 0 -1  0  3  0  0  1  2  2  0 -1  0  2  3 -1  1 -1  4 -1  0 -1 -1  5 -1
 -1  0  4  1  3  5  2  2  3 -1 -1 -1 -1  1  6  0  3 -1  0 -1 -1 -1  3 -1
  1 -1 -1  0  2 -1 -1  2  0 -1 -1  3  3  1  1 -1 -1  6 -1 -1  2  3  0  2
  1  3  2  3 -1  0  1 -1 -1  0  0  2 -1  5  1  1  0 -1 -1 -1  1  0  1 -1
  4  3  1  2 -1  0  3  0  4  1 -1  5 -1 -1  2  6 -1  0 -1  3  3 -1  3  4
  1  2 -1 -1 -1 -1  5  2 -1  2  0  3  0 -1  1 -1  0 -1 -1  3  3  2  0  1
  3  0  5  0  3  3  0 -1 -1 -1 -1  3 -1  3 -1 -1  0  3  3 -1 -1  5  6 -1
 -1  3]


In [167]:
#KDE for iridium garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1500)
model.fit(data_garbage_iridium)
log_dens = model.score_samples(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_dens})])
fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()

In [168]:
#KDE for cosmos garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1000)
model.fit(data_garbage_cosmos)
log_denss = model.score_samples(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_denss})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()


In [169]:
result_iridium['Cluster'] = pred
result_iridium['Density'] = log_dens
result_iridium.to_csv('../Data/result_iridium.csv', index= False)
result_iridium

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,IRIDIUM 33,1997-051C,2024-03-15T03:57:12.554496,37.454356,-517.294451,-7150.266054,0,-27.254635
1,IRIDIUM 33 DEB,1997-051L,2024-03-14T17:47:59.202816,6107.815304,-3679.558554,-337.136238,-1,-28.847182
2,IRIDIUM 33 DEB,1997-051N,2024-03-15T04:39:16.727904,590.592563,-1083.968406,-7055.841393,0,-27.420027
3,IRIDIUM 33 DEB,1997-051P,2024-03-14T20:44:27.452832,-2130.006345,927.526528,6742.635598,3,-27.672178
4,IRIDIUM 33 DEB,1997-051Q,2024-03-14T15:11:01.815360,-130.410782,-546.825468,-7092.620476,0,-27.240286
...,...,...,...,...,...,...,...,...
165,IRIDIUM 33 DEB,1997-051ABX,2024-03-14T23:53:25.857024,-931.779097,-6550.183170,2550.246793,5,-28.515859
166,IRIDIUM 33 DEB,1997-051ACD,2024-03-14T07:15:03.155904,974.426821,5028.714984,4869.331431,6,-28.124507
167,IRIDIUM 33 DEB,1997-051ACE,2024-03-11T22:52:08.279904,-4330.987886,4546.253020,-3381.394169,-1,-28.242389
168,IRIDIUM 33 DEB,1997-051ACG,2024-03-14T13:33:50.161824,6076.635485,2173.404857,3091.019318,-1,-28.906674


In [170]:
result_cosmos['Cluster'] = predd
result_cosmos['Density'] = log_denss
result_cosmos.to_csv('../Data/result_cosmos.csv', index= False)
result_cosmos

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,COSMOS 2251,1993-036A,2024-03-14T21:31:51.966336,-4931.626965,-4517.122470,-2556.738608,-1,-28.042205
1,COSMOS 2251 DEB,1993-036E,2024-03-14T21:50:15.684000,-5404.417337,-4348.594356,1757.855398,0,-28.128777
2,COSMOS 2251 DEB,1993-036F,2024-03-14T18:05:33.239616,5357.145811,-2368.563469,4074.086565,-1,-28.286374
3,COSMOS 2251 DEB,1993-036G,2024-03-14T08:58:39.399168,3838.897765,-2043.124705,5526.069115,1,-27.989453
4,COSMOS 2251 DEB,1993-036H,2024-03-14T21:23:40.216416,4497.423444,-1050.758684,-5457.088636,-1,-27.911136
...,...,...,...,...,...,...,...,...
769,COSMOS 2251 DEB,1993-036BXS,2024-03-04T05:57:55.018944,-4570.122729,-5299.813024,-2623.325863,-1,-28.141843
770,COSMOS 2251 DEB,1993-036BXY,2024-03-14T01:11:36.902688,4854.509853,-3031.932595,4257.712205,-1,-28.343176
771,COSMOS 2251 DEB,1993-036BYA,2024-03-14T03:53:33.007776,-2361.647368,652.670466,-6619.418741,8,-27.437786
772,COSMOS 2251 DEB,1993-036BYD,2024-03-09T04:50:01.458528,2726.956482,4752.615689,-4955.250403,36,-28.384666


In [171]:
result_satellite.to_csv('../Data/result_satellite.csv', index= False)
result_satellite

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z
0,HBTSS-SV2,2024-028A,2024-03-14T20:49:05.202912,6651.129175,479.204659,3135.045204
1,RAPTOR4,2024-028B,2024-03-14T20:47:39.831936,6735.227281,1062.264739,2793.348293
2,RAPTOR1,2024-028C,2024-03-14T20:46:13.850112,6761.567993,1644.245135,2423.220515
3,RAPTOR3,2024-028D,2024-03-14T22:29:38.631552,6724.164442,2253.166255,2001.039876
4,RAPTOR2,2024-028E,2024-03-14T20:43:13.249056,6624.874939,2812.316443,1580.608275
...,...,...,...,...,...,...
199,STARLINK-31283,2024-044Y,2024-03-15T04:00:00.999936,-3297.713167,-5600.410236,1775.614370
200,EXO-0,2023-174DK,2024-03-14T21:03:32.576832,-6150.293346,3136.658001,300.775562
201,2024-047A,2024-047A,2024-03-15T04:34:59.235168,3595.926096,4561.030996,-3797.452830
202,2024-047B,2024-047B,2024-03-13T09:40:18.197760,4179.917376,4932.989824,-2511.263106


In [172]:
result_iridium.groupby('Cluster')['Density'].mean().abs()

Cluster
-1    28.583792
 0    27.584511
 1    28.136223
 2    27.998820
 3    27.339365
 4    28.247988
 5    28.402587
 6    28.161088
Name: Density, dtype: float64

In [173]:
result_cosmos.groupby('Cluster')['Density'].mean().abs()

Cluster
-1     28.309507
 0     28.235318
 1     27.999518
 2     27.548282
 3     27.527217
 4     27.825880
 5     27.978860
 6     28.029793
 7     27.938959
 8     27.334003
 9     27.416676
 10    27.745313
 11    28.112430
 12    27.931522
 13    27.718403
 14    27.680233
 15    27.798786
 16    27.849147
 17    27.994474
 18    28.194809
 19    27.948288
 20    27.728510
 21    27.900101
 22    28.408534
 23    28.189136
 24    28.148853
 25    28.303787
 26    28.338294
 27    28.076550
 28    27.703009
 29    28.049490
 30    27.978596
 31    28.119948
 32    27.889669
 33    28.413676
 34    28.095254
 35    28.227632
 36    28.355548
 37    27.950286
 38    28.238440
 39    28.214673
 40    28.203306
 41    28.388127
 42    27.819737
 43    28.508247
Name: Density, dtype: float64

In [174]:
print(data_garbage_iridium)

[[   37.45435583  -517.29445148 -7150.26605386]
 [ 6107.81530398 -3679.55855376  -337.13623824]
 [  590.59256339 -1083.96840564 -7055.84139301]
 [-2130.0063449    927.52652848  6742.63559784]
 [ -130.4107821   -546.82546805 -7092.62047586]
 [-1840.48663807  2107.1826115  -6587.6537454 ]
 [ -548.75180311 -6353.79809118 -3307.521398  ]
 [-2831.30376032  6512.4754035   -237.64376104]
 [-3479.38151457  5319.98222166  3168.17548185]
 [   88.66452664  -728.85342363 -7098.26038705]
 [ 4225.2749023   2712.10451622  4927.18860232]
 [-2804.93181926  -454.75969485 -6703.74304792]
 [-1733.60257291  6789.71324097  -582.08501754]
 [ -579.03553329  1314.68694343  6638.74654825]
 [ 5964.02365364 -3423.49326032  1929.83852288]
 [-2080.839467   -6396.2576955  -1836.02860799]
 [-1140.55303411  3375.48905424  6167.03962247]
 [-4132.24614719 -4778.69364223 -2451.48177636]
 [ 1732.56831692   114.0200268  -6985.75868568]
 [-1631.89010072  2970.80618202 -6228.67822026]
 [ 6105.57232443 -3100.61980092 -2124.71

In [175]:
import csv
import pandas as pd

distances_s_i = []
for i, garbage in enumerate(data_garbage_iridium):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_i.append((i, j, distance))

distances = pd.DataFrame(distances_s_i, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_i.csv', index=False)

       garbage_index  satellite_index      distance
0                  0                0  12268.713574
1                  0                1  12092.585980
2                  0                2  11896.957706
3                  0                3  11667.645201
4                  0                4  11432.785540
...              ...              ...           ...
34675            169              199   8985.661739
34676            169              200  11696.358173
34677            169              201  11706.938463
34678            169              202  10854.808029
34679            169              203  12759.234664

[34680 rows x 3 columns]


In [176]:
import csv
import pandas as pd

distances_s_c = []
for i, garbage in enumerate(data_garbage_cosmos):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_c.append((i, j, distance))

distances = pd.DataFrame(distances_s_c, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_c.csv', index=False)

        garbage_index  satellite_index      distance
0                   0                0  13839.072495
1                   0                1  13995.302050
2                   0                2  14124.208011
3                   0                3  14229.111970
4                   0                4  14296.539800
...               ...              ...           ...
157891            773              199   6487.825389
157892            773              200  10514.778376
157893            773              201  11093.337813
157894            773              202  11853.379983
157895            773              203   2745.022323

[157896 rows x 3 columns]


In [177]:

# # DBSCAN for cosmos garbage
# with open('../Data/cosmos.csv') as f:
#         lines = omm.parse_csv(f)
#         fields = [field for field in lines]
# 
# Cosmos_G = []
# result_cosmos = []
# 
# for field in fields:
#     sat = Satrec()
#     satellite = omm.initialize(sat, field)
#     jd, fr = xd, xr
#     e, r, v = sat.sgp4(jd, fr)
#     Cosmos_G.append(r)
#     result_cosmos.append([field['OBJECT_ID'],field['EPOCH'], *r])
# 
# data_garbage_cosmos = np.array(Cosmos_G)
# data_garbage_cosmos = data_garbage_cosmos[~np.isnan(data_garbage_cosmos).any(axis=1)]
# 
# result_cosmos = pd.DataFrame(result_cosmos,columns=['OBJECT_ID','EPOCH','x','y','z']).dropna()
# 
# 
# 
# # model = DBSCAN(eps=800, min_samples=4)
# # model.fit_predict(data_garbage_cosmos)
# model=mix.GaussianMixture(n_components=10,covariance_type="full")
# model.fit_predict(data_garbage_cosmos)
# predd = model.fit_predict(data_garbage_cosmos)
# 
# fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
#                                    mode='markers', marker={"size":12, "color":predd})])
# fig.show()


In [178]:
# import numpy as np
# import plotly.graph_objects as go
# from scipy.stats import multivariate_normal
# 
# # Ініціалізація параметрів
# n_components = 3
# mu = np.random.rand(n_components, 3)  # Випадкова ініціалізація середніх значень
# sigma = [np.eye(3) for _ in range(n_components)]  # Ініціалізація одиничними коваріаційними матрицями
# pi = np.ones(n_components) / n_components  # Рівномірні ваги
# z = np.zeros((len(data_garbage_cosmos), n_components))
# 
# def likelihood(x, mu, sigma):
#     return multivariate_normal(mean=mu, cov=sigma).pdf(x)
# 
# # Алгоритм EM
# for step in range(100):
#     # Крок E
#     for k in range(n_components):
#         z[:, k] = pi[k] * likelihood(data_garbage_cosmos, mu[k], sigma[k])
#     z /= z.sum(axis=1, keepdims=True)
#     
#     # Крок M
#     for k in range(n_components):
#         weight = np.sum(z[:, k])
#         mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_cosmos, axis=0) / weight
#         diff = data_garbage_cosmos - mu[k]
#         cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
#         if np.linalg.matrix_rank(cov) < 3:  # Перевірка на оборотність
#             cov = np.linalg.pinv(cov)  # Застосування псевдообернення
#         sigma[k] = cov
#         pi[k] = weight / len(data_garbage_cosmos)
# 
# # Візуалізація результату
# fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data')])
# fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
# fig.show()


In [179]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import multivariate_normal

# Filter out rows with inf or NaN values
data_garbage_cosmos = np.array([row for row in data_garbage_cosmos if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_cosmos), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_cosmos, mu[k], sigma[k])
    z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_cosmos, axis=0) / weight
        diff = data_garbage_cosmos - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_cosmos) + n_components * 1e-6), 0, 1)

# Create a list of colors based on the number of components
colors = ['rgb({}, {}, {})'.format(*[int(i * 255) for i in (np.random.rand(3) + np.array([0.2, 0.2, 0.2]))]) for _ in range(n_components)]

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data',marker=dict(color=[colors[i] for i in np.argmax(z, axis=1)], opacity=0.8, size=8))])

fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [180]:
print(np.any(np.isnan(data_garbage_cosmos)))
print(np.any(np.isinf(data_garbage_cosmos)))


False
False


In [181]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import multivariate_normal

# Filter out rows with inf or NaN values
data_garbage_cosmos = np.array([row for row in data_garbage_cosmos if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_cosmos), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_cosmos, mu[k], sigma[k])
        z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_cosmos, axis=0) / weight
        diff = data_garbage_cosmos - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_cosmos) + n_components * 1e-6), 0, 1)

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data')])
fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [182]:
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=20)
kmeans.fit(data_garbage_cosmos)

labels = kmeans.labels_
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":pred})])

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data', marker=dict(size=8, color=labels, colorscale='Viridis', showscale=True))])


fig.add_trace(go.Scatter3d(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], z=kmeans.cluster_centers_[:, 2],
                          mode='markers', name='Cluster Centers', marker=dict(size=12, color='red', opacity=0.7)))

fig.show()

# нечіткий k-means